# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [2]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [16]:
%store -r numerosKalaba typeKalaba
numerosKalaba=[5]
# numerosKalaba=[18]
print numerosKalaba
typeKalaba="Kalaba"
%store numerosKalaba 
%store typeKalaba

[5]
Stored 'numerosKalaba' (list)
Stored 'typeKalaba' (str)


In [4]:
home = expanduser("~")

if typeKalaba!="Kanonik":
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
    serie=repertoire+"23-"
    nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]
else:
    repertoire=home+"/Library/Mobile Documents/com~apple~CloudDocs/Downloads/"
    serie=repertoire+"23-Kanoniks/"
    nomsKalabas=[serie+"Kanonik-%02d-ok/"%num for num in numerosKalaba]

# Traitement

In [5]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [6]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    
    lexiqueTestPrep=getExemple(stems["PREP"]).lower()
    lexiqueTestNoun=getExemple(stems["NOM"]).lower()
    lexiqueTestHyper=getExemple(stems["ADJ"]).lower()
    
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    # classesNom=PFM.lexique.formeLexeme[mot.lower()][0].split('.')[1:]
    # [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/23-K2/
NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,A
head stems,NOM,C
head stems,NOM,B
head stems,VER
head stems,DET


### Bilan

In [7]:
stems["NOM"]

{'A': {'N1': {'NuDin': ['sorcier',
    u'sorci\xe8re',
    'sorciers',
    u'sorci\xe8res'],
   'bagul': ['excuse', 'excuses'],
   'debom': ['gargouille', 'gargouilles'],
   'fevaT': ['villageois.HYPER', 'villagEOIs', 'villagEOIS'],
   'fidaS': ['nouvelle-N', 'nouvELLE', 'nouvELLEs'],
   'fidiT': ['hurlement', 'hurlements'],
   'geDap': ['maison', 'maisons'],
   'katiSa': ['Katisha', 'Katishas'],
   'kovfefe': ['cri', 'cris'],
   'kuboT': ['crapaud', 'crapauds'],
   'momad': ['plaine', 'plaines'],
   'nabup': ['marais', 'maraiS'],
   'nerul': ['combat', 'combats'],
   'padat': ['chat', 'chats'],
   'peZus': [u'col\xe8re', u'col\xe8res'],
   'ruvip': ['disparition', 'disparitions'],
   'ruzos': ['dragon', 'dragons'],
   'tudal': ['loup', 'loups'],
   'vonud': ['lance', 'lances'],
   'wiraw': [u'ma\xeetre', u'ma\xeetresses', u'ma\xeetresse', u'ma\xeetres']},
  'N2': {'NefTit': [u'proximit\xe9', u'proximit\xe9s'],
   'deduf': ['arbre', 'arbres'],
   'dobup': [u'infirmi\xe8re', u'infirmi\x

In [8]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
# print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

suivant
pour


In [9]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=A',
 u'VER, Temps=PRS, Pers=3Sg, Genre=B',
 u'VER, Temps=PRS, Pers=3Sg, Genre=C',
 u'VER, Temps=PRS, Pers=3Du, Genre=A',
 u'VER, Temps=PRS, Pers=3Du, Genre=B',
 u'VER, Temps=PRS, Pers=3Du, Genre=C',
 u'VER, Temps=PRS, Pers=3Pl, Genre=A',
 u'VER, Temps=PRS, Pers=3Pl, Genre=B',
 u'VER, Temps=PRS, Pers=3Pl, Genre=C',
 u'VER, Temps=PST, Pers=3Sg, Genre=A',
 u'VER, Temps=PST, Pers=3Sg, Genre=B',
 u'VER, Temps=PST, Pers=3Sg, Genre=C',
 u'VER, Temps=PST, Pers=3Du, Genre=A',
 u'VER, Temps=PST, Pers=3Du, Genre=B',
 u'VER, Temps=PST, Pers=3Du, Genre=C',
 u'VER, Temps=PST, Pers=3Pl, Genre=A',
 u'VER, Temps=PST, Pers=3Pl, Genre=B',
 u'VER, Temps=PST, Pers=3Pl, Genre=C']

In [10]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [11]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[('X+Z', 'Temps=PRS')]

In [12]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [13]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

coup.B.N2 DepaT
histoire.C.N2 jafod
six rugik
transformer.V1 goSara
envahir.V1 busim
blessure.C.N2 zebeb
mort.B.N2 tezeb
enlever.V1 nugef
acheter.V2 bazon
esprit.C.N2 kebev
donner.V2 liDaZ
combattant.C.N2 dizuT
souris.C.N1 podaD
manoir.B.N2 bedin
sort.C.N2 sinij
viande.A.N2 kaduT
livre.B.N1 baZeb
coyote.B.N2 lanur
dernier-N.B.N1 Nanub
pleurer-Act.V2 ZeTug
noir todib
lever-N.B.N1 Sonul
être.V2 gonat
lever.V1 livel
PRO S
table.B.N1 fibeZ
Agathos.C.N2 agaTos
partir.V1 duneN
raconter.V2 duzopi
pleurer.V2 zapal
jardiner.V1 puzef
Hoderi.B.N2 oderi
visage.B.N1 Subis
lance.A.N1 vonud
tornade.A.N2 genis
compter.V1 guvit
légende.C.N1 nigeN
sur feta
cuisine.B.N2 paraZ
vers pudut
proximité.A.N2 NefTit
enfermer.V2 pujot
villageois.C.N2 nezop
arrêter.V1 girari
de wu
noyer.V1 febiN
marais.A.N1 nabup
découvrir.V2 keNiz
couteau.C.N1 pized
combat.A.N1 nerul
cinq Nivali
ressusciter.V1 nugeS
disparition.A.N1 ruvip
souffrance.A.N2 zuZev
infirmière.A.N2 dobup
obscurité.C.N1 tinuk
Elphaba.C.N2 elfaba
bas koD

# Conclusion

In [14]:
ding()